In [ ]:
%reload_ext autoreload
%autoreload 2

from configData import configVars
from transcriptLoader import TranscriptData

# videoNames = [
#     "New Quizzes Video",
#     "Rearrange Playlist video",
#     "IMSE 514 Presentation",
#     "New Google Assignments in Canvas",
#     "Piazza Introduction Workshop",
# ]

config = configVars()
config.videoToUse = "New Quizzes Video"
config.setFromEnv()


videoData = TranscriptData(config)

In [ ]:
from transcriptLoader import retrieveTranscript

videoData = retrieveTranscript(config, overwrite=True)
videoData.printTranscript()

In [2]:
from topicExtractor import retrieveTopics

topicModeller = retrieveTopics(config, videoData, overwrite=True)
topicModeller.printTopics()

In [ ]:
from questionGenerator import retrieveQuestions

questionData = retrieveQuestions(config, topicModeller, videoData, overwrite=True)
questionData.printQuestions()